## Fetch the github topics page

In [1]:
import requests
url = 'https://github.com/topics'
responce = requests.get(url)
responce.status_code

200

### Extract the HTML code

In [2]:
page_content = responce.text


In [3]:
with open ('webpage.html','w') as f:
  f.write(page_content)

### use  BeautifulSoup Library

In [4]:
from bs4 import BeautifulSoup
doc = BeautifulSoup(page_content,'html.parser')

### Get topic titles

In [5]:
topic_title = doc.find_all('p',class_='f3 lh-condensed mb-0 mt-1 Link--primary')

In [6]:
topic_titles=[topic.text for topic in topic_title]
topic_titles[:5]

['3D', 'Ajax', 'Algorithm', 'Amp', 'Android']

### Get Topics description

In [7]:
topic_decription = doc.find_all('p',class_='f5 color-fg-muted mb-0 mt-1')

In [8]:
topic_docs =[doc.text for doc in topic_decription]
topic_docs[0]

'\n          3D refers to the use of three-dimensional graphics, modeling, and animation in various industries.\n        '

In [13]:
for i in range(len(topic_docs)):
    topic_docs[i] = topic_docs[i].strip()
topic_docs[0]

'3D refers to the use of three-dimensional graphics, modeling, and animation in various industries.'

### Get topics url

In [15]:
link_tag = doc.find_all('a',class_='no-underline flex-1 d-flex flex-column')
topic_urls = ['https://github.com'+link['href'] for link in link_tag]
topic_urls[:5]

['https://github.com/topics/3d',
 'https://github.com/topics/ajax',
 'https://github.com/topics/algorithm',
 'https://github.com/topics/amphp',
 'https://github.com/topics/android']

## Putting all the informatations in a csv file

In [16]:
import pandas as pd
top_topics = pd.DataFrame({'Title':topic_titles,'Description':topic_docs,'URL':topic_urls})
top_topics.to_csv('top_topics.csv',index=None)


In [17]:
top_topics.head(5)

,Title,Description,URL
0,3D,3D refers to the use of three-dimensional grap...,https://github.com/topics/3d
1,Ajax,Ajax is a technique for creating interactive w...,https://github.com/topics/ajax
2,Algorithm,Algorithms are self-contained sequences that c...,https://github.com/topics/algorithm
3,Amp,Amp is a non-blocking concurrency library for ...,https://github.com/topics/amphp
4,Android,Android is an operating system built by Google...,https://github.com/topics/android


### Get the top 20 reositories in each topic

### **the extracted information /name / author /stars /url**

In [66]:
def topic_url(url):
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception('Failed to load page {}'.format(url))
    doc = BeautifulSoup(response.text, 'html.parser')

    # Get repository names
    repository_names = doc.find_all('a', class_='Link text-bold wb-break-word')
    repository_names = [name.text.strip() for name in repository_names]

    # Get repository authors
    repository_author = doc.find_all('h3', class_='f3 color-fg-muted text-normal lh-condensed')
    repository_auth = []
    for author in repository_author:
        author_text = author.text.split('/')[0].strip()
        repository_auth.append(author_text)

    # Get repository stars
    repository_stars = doc.find_all('span', class_='Counter js-social-count')
    repository_star = []
    for star in repository_stars :  # Assuming there are at least 5 star elements
        star_text = star.text.strip()
        repository_star.append(star_text)

    # Create repository links
    rep_link = ["https://github.com/" + repository_auth[i] + "/" + repository_names[i] for i in range(len(repository_names))]

    # Create DataFrame
    df = pd.DataFrame({
        'Repository Name': repository_names,
        'Repository Author': repository_auth,
        'Stars': repository_star,
        'Link': rep_link
    })
    return df

### Putting all together

In [72]:
def topic_repositories(url):
    topic_df = topic_url(url)
    df_name = url.replace('https://github.com/topics/','')+'_Top_Repositories.csv'
    topic_df.to_csv(df_name,index=None)
    return topic_df

## Get the csv files

In [73]:
for url in top_topics['URL'].values:
    df = topic_repositories(url)

